In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet, EfficientNetB0, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import AdamW, SGD
from tensorflow.keras.callbacks import CosineDecayRestarts, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.utils import class_weight
import cv2
import albumentations as A
from keras_tuner import RandomSearch
import streamlit as st
import os
import io
from PIL import Image


ImportError: Traceback (most recent call last):
  File "C:\Users\joane\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
# Constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 4  # Glioma, meningioma, pituitary, no tumor
DATASET_PATH = 'path/to/dataset'  # Update this
TRAIN_DIR = os.path.join(DATASET_PATH, 'train')
VAL_DIR = os.path.join(DATASET_PATH, 'validation')
TEST_DIR = os.path.join(DATASET_PATH, 'test')


In [ ]:
# Data Augmentation
def get_augmentation_pipeline():
    return A.Compose([
        A.Rotate(limit=20, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomZoom(zoom_limit=0.2, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.3),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])


In [ ]:
# Data Generators
def create_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        preprocessing_function=lambda x: get_augmentation_pipeline()(image=x)['image']
    )
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True
    )
    val_generator = val_datagen.flow_from_directory(
        VAL_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
    )
    test_generator = val_datagen.flow_from_directory(
        TEST_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
    )
    
    class_weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(train_generator.classes),
        y=train_generator.classes
    )
    class_weights = dict(enumerate(class_weights))
    
    return train_generator, val_generator, test_generator, class_weights


In [ ]:
# Model Creation
def create_model(model_name, num_classes, learning_rate=1e-3):
    if model_name == 'MobileNet':
        base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    elif model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    else:
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    base_model.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=AdamW(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model, base_model


In [ ]:
# Fine-tuning function
def fine_tune_model(model, base_model, train_generator, val_generator, class_weights, learning_rate=1e-4):
    total_layers = len(base_model.layers)
    unfreeze_layers = int(total_layers * 0.2)
    for layer in base_model.layers[-unfreeze_layers:]:
        layer.trainable = True
    
    lr_schedule = CosineDecayRestarts(initial_learning_rate=learning_rate, first_decay_steps=10)
    model.compile(
        optimizer=SGD(learning_rate=lr_schedule, momentum=0.9),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    history = model.fit(
        train_generator,
        epochs=EPOCHS // 2,
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
    )
    
    return history


In [ ]:
# Hyperparameter Tuning
def build_model_for_tuning(hp):
    model, _ = create_model('MobileNet', NUM_CLASSES, hp.Float('learning_rate', 1e-4, 1e-2, sampling='log'))
    model.layers[-2].rate = hp.Float('dropout_rate', 0.3, 0.7, step=0.1)
    return model

def tune_hyperparameters(train_generator, val_generator):
    tuner = RandomSearch(
        build_model_for_tuning,
        objective='val_accuracy',
        max_trials=5,
        executions_per_trial=1,
        directory='tuner_dir',
        project_name='brain_tumor'
    )
    tuner.search(
        train_generator,
        epochs=10,
        validation_data=val_generator,
        callbacks=[EarlyStopping(patience=3)]
    )
    return tuner.get_best_hyperparameters()[0]


In [ ]:
# Grad-CAM
def get_gradcam_heatmap(model, img_array, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy()

def plot_gradcam(img_path, model, last_conv_layer_name, class_names):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array / 255.0, axis=0)
    
    heatmap = get_gradcam_heatmap(model, img_array, last_conv_layer_name)
    heatmap = cv2.resize(heatmap, (IMG_WIDTH, IMG_HEIGHT))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    superimposed_img = heatmap * 0.4 + img_array[0] * 255.0
    superimposed_img = superimposed_img / np.max(superimposed_img)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title('Original Image')
    plt.subplot(1, 2, 2)
    plt.imshow(superimposed_img)
    plt.title(f'Grad-CAM: {class_names[np.argmax(model.predict(img_array))]}')
    plt.show()


In [ ]:
# Model Evaluation
def evaluate_model(model, test_generator, class_names):
    predictions = model.predict(test_generator)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_generator.classes

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))
    
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    y_true_bin = tf.keras.utils.to_categorical(y_true, NUM_CLASSES)
    plt.figure(figsize=(8, 6))
    for i in range(NUM_CLASSES):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], predictions[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves')
    plt.legend()
    plt.show()
